<a href="https://colab.research.google.com/github/alerosenberg/rais_ale/blob/main/Importar_RAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transferir bases da RAIS para o Google Drive

In [ ]:
# Acessar o site abaixo:
# ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/
# Escolher os anos e os estados desejados
# Descompatar o arquivo no computador
# Transferir o txt para o Google Drive
# Cada arquivo (utilizo como referência a RAIS de cada ano para o estado de SP) ocupa cerca de 6GB

Importar arquivos do Google Drive para o Google Colab

In [5]:
#STEP-1: Import libraries
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [11]:
#STEP-2: Autheticate E-Mail ID
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [12]:
# STEP-3: Get File from Drive using file-ID
# Após 'id': na linha abaixo é necessário trocar pelo caminho do arquivo desejado 
# O caminho pode ser obtido clicando com o botão direito sobre o arquivo e em Gerar Link
downloaded = drive.CreateFile({'id':'1KaNuUKq_8vXMcS4K2Fpb6ceZaTJmzTid'})
# Abaixo devem ser indicados os arquivos desejados (que já devem estar presentes no Drive)
downloaded.GetContentFile('SP2006.csv') 

Instalando bibliotecas

In [ ]:
!pip install wget 
!pip install py7zr
!pip install dask
!pip install dask[dataframe]
#!pip install sys
!pip install tqdm
#!pip install os
!pip install pandas
!pip install numpy
!pip install scipy
!pip install matplotlib
!pip install seaborn 
!pip install deflatebr

Importando bibliotecas

In [39]:
import wget
import py7zr
import dask 
import dask.dataframe as dd
import sys
from tqdm import tqdm
import py7zr
import os
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from statsmodels.graphics.factorplots import interaction_plot
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import matplotlib.pyplot as plt
import seaborn as sns
from deflatebr import deflate

Importando dados

In [64]:
#STEP-4: Read File
#3.1 Read file as panda dataframe
base = dd.read_csv('SP2006.csv', sep=';', encoding="latin1", assume_missing=True,
                        dtype={'CBO Ocupação 2002': 'object',
                                'Faixa Remun Dezem (SM)': 'object',
                                'Faixa Tempo Emprego': 'object',
                                'Faixa Etária': 'object',
                                'Bairros SP': 'object',
                                'Distritos SP': 'object',
                                'Bairros RJ': 'object',
                                'Raça Cor': 'object',
                                'Natureza Jurídica': 'object',
                                'CNAE 2.0 Subclasse': 'object',
                                'Tipo Defic': 'object',
                                'CNAE 2.0 Classe': 'object',
                                'CNAE 95 Classe': 'object'})                 

Visualizar dados

In [45]:
base.columns


Index(['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'Causa Afastamento 1',
       'Causa Afastamento 2', 'Causa Afastamento 3', 'Motivo Desligamento',
       'CBO Ocupação 2002', 'CNAE 2.0 Classe', 'CNAE 95 Classe',
       'Distritos SP', 'Vínculo Ativo 31/12', 'Faixa Etária',
       'Faixa Hora Contrat', 'Faixa Remun Dezem (SM)',
       'Faixa Remun Média (SM)', 'Faixa Tempo Emprego',
       'Escolaridade após 2005', 'Qtd Hora Contr', 'Idade',
       'Ind CEI Vinculado', 'Ind Simples', 'Mês Admissão', 'Mês Desligamento',
       'Mun Trab', 'Município', 'Nacionalidade', 'Natureza Jurídica',
       'Qtd Dias Afastamento', 'Raça Cor', 'Regiões Adm DF',
       'Vl Remun Dezembro Nom', 'Vl Remun Dezembro (SM)', 'Vl Remun Média Nom',
       'Vl Remun Média (SM)', 'CNAE 2.0 Subclasse', 'Sexo Trabalhador',
       'Tamanho Estabelecimento', 'Tempo Emprego', 'Tipo Admissão',
       'Tipo Estab', 'Tipo Estab.1', 'Tipo Vínculo'],
      dtype='object')

In [72]:
base.head(100)

,Bairros SP,Bairros Fortaleza,Bairros RJ,Causa Afastamento 1,Causa Afastamento 2,Causa Afastamento 3,Motivo Desligamento,CBO Ocupação 2002,CNAE 2.0 Classe,CNAE 95 Classe,Distritos SP,Vínculo Ativo 31/12,Faixa Etária,Faixa Hora Contrat,Faixa Remun Dezem (SM),Faixa Remun Média (SM),Faixa Tempo Emprego,Escolaridade após 2005,Qtd Hora Contr,Idade,Ind CEI Vinculado,Ind Simples,Mês Admissão,Mês Desligamento,Mun Trab,Município,Nacionalidade,Natureza Jurídica,Qtd Dias Afastamento,Raça Cor,Regiões Adm DF,Vl Remun Dezembro Nom,Vl Remun Dezembro (SM),Vl Remun Média Nom,Vl Remun Média (SM),CNAE 2.0 Subclasse,Sexo Trabalhador,Tamanho Estabelecimento,Tempo Emprego,Tipo Admissão,Tipo Estab,Tipo Estab.1,Tipo Vínculo
0,0225,0.0,0000,{ñ,{ñ,{ñ,11.0,514210,81125,70408,0028,0.0,07,6.0,00,04,07,1.0,44.0,61.0,0.0,0.0,09,8.0,355030,355030.0,10.0,3085,0.0,02,{ñ c,"0000000000,00","000000,00","0000000866,27","000002,60",8112500,1.0,3.0,"83,7",0.0,1.0,CNPJ,10.0
1,0672,0.0,0000,{ñ,{ñ,{ñ,21.0,783210,46338,51330,0081,0.0,03,6.0,00,03,01,1.0,44.0,22.0,0.0,1.0,09,11.0,355030,355030.0,10.0,2062,0.0,09,{ñ c,"0000000000,00","000000,00","0000000572,70","000001,63",4633801,1.0,2.0,"2,3",2.0,1.0,CNPJ,10.0
2,9999,0.0,0000,{ñ,{ñ,{ñ,11.0,622110,49299,60259,9999,0.0,05,6.0,03,02,02,1.0,44.0,34.0,0.0,1.0,07,12.0,0000-1,353510.0,10.0,2135,0.0,02,{ñ c,"0000000372,34","000001,06","0000000502,13","000001,43",4929902,1.0,2.0,"5,3",2.0,1.0,CNPJ,70.0
3,9999,0.0,0000,{ñ,{ñ,{ñ,11.0,717020,41204,45217,9999,0.0,05,6.0,00,03,06,1.0,44.0,37.0,0.0,0.0,01,3.0,0000-1,354100.0,10.0,2062,0.0,02,{ñ c,"0000000000,00","000000,00","0000000587,25","000001,95",4120400,1.0,5.0,"37,7",0.0,1.0,CNPJ,10.0
4,9999,0.0,0000,{ñ,{ñ,{ñ,31.0,721215,25314,28312,9999,0.0,03,6.0,00,04,02,1.0,44.0,21.0,0.0,0.0,12,5.0,0000-1,354870.0,10.0,2062,0.0,02,{ñ c,"0000000000,00","000000,00","0000000815,71","000002,61",2531401,1.0,3.0,"5,0",0.0,1.0,CNPJ,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,{ñ class},0.0,0000,{ñ,{ñ,{ñ,12.0,517405,78205,74500,0072,0.0,03,6.0,03,02,01,1.0,44.0,23.0,0.0,0.0,09,12.0,355030,355030.0,10.0,2062,0.0,02,{ñ c,"0000000450,54","000001,28","0000000450,55","000001,28",7820500,1.0,7.0,"2,9",2.0,1.0,CNPJ,50.0
96,1668,0.0,0000,{ñ,{ñ,{ñ,21.0,517410,81125,70408,0053,0.0,06,6.0,00,04,03,1.0,44.0,40.0,0.0,0.0,02,9.0,355030,355030.0,10.0,3085,0.0,02,{ñ c,"0000000000,00","000000,00","0000000953,30","000002,80",8112500,1.0,2.0,"7,6",2.0,1.0,CNPJ,10.0
97,1791,0.0,0000,{ñ,{ñ,{ñ,11.0,514225,81125,70408,0021,0.0,05,6.0,00,02,04,1.0,44.0,35.0,0.0,0.0,10,8.0,355030,355030.0,10.0,3085,0.0,08,{ñ c,"0000000000,00","000000,00","0000000434,70","000001,37",8112500,1.0,2.0,"21,5",0.0,1.0,CNPJ,10.0
98,9999,0.0,0000,{ñ,{ñ,{ñ,21.0,724110,42995,45292,9999,0.0,07,6.0,00,06,01,1.0,44.0,54.0,0.0,0.0,08,9.0,0000-1,350950.0,10.0,2062,0.0,08,{ñ c,"0000000000,00","000000,00","0000001503,62","000004,29",4299599,1.0,7.0,"0,7",2.0,1.0,CNPJ,10.0


In [ ]:
# # Corrige grau de escolaridade para obter valor 0
#     dados['Escolaridade após 2005'] = pd.to_numeric(dados['Escolaridade após 2005'], errors="coerce")
#     dados['Escolaridade após 2005'] -= 1
#     # Coloca label em sexo
#     dados['Sexo Trabalhador'] = pd.to_numeric(dados['Sexo Trabalhador'],errors="coerce")
#     dados.loc[dados['Sexo Trabalhador'] == 1, 'Sexo Trabalhador'] = "Masculino"
#     dados.loc[dados['Sexo Trabalhador'] == 2, 'Sexo Trabalhador'] = "Feminino"
#     # Filtra apenas quem Raça Cor não é 99
#     dados['Raça Cor'] = pd.to_numeric(dados['Raça Cor'], errors="coerce")
#     dados = dados.loc[dados["Raça Cor"] != 99,:]
#     # Coloca label em raça
#     dados.loc[dados["Raça Cor"] == 1, 'Raça Cor'] = "Indígena"
#     dados.loc[dados["Raça Cor"] == 2, 'Raça Cor'] = "Branca"
#     dados.loc[dados["Raça Cor"] == 4, 'Raça Cor'] = "Preta"
#     dados.loc[dados["Raça Cor"] == 6, 'Raça Cor'] = "Amarela"
#     dados.loc[dados["Raça Cor"] == 8, 'Raça Cor'] = "Parda"
#     dados.loc[dados["Raça Cor"] == 9, 'Raça Cor'] = "Não identificada"
#     # Filtra apenas quem tem salário maior que 0 e idade de 18 a 80 anos
#     dados['Idade'] = pd.to_numeric(dados['Idade'], errors="coerce")
#     dados = dados.loc[(dados['Vl Remun Média Nom'] > 0) & 
#                         (dados['Idade'] > 17) & 
#                         (dados['Idade'] < 81), :]

#     # Cria variável binária Brasileiro
#     dados['Nacionalidade'] = pd.to_numeric(dados['Nacionalidade'], errors="coerce")
#     dados['Brasileiro'] = dados['Nacionalidade'].apply(lambda x: 1 if (x == 10) else 0)
